**BIBLIOTECAS** 📚

In [ ]:
!pip install PyGithub
import pandas as pd
import unicodedata
from datetime import datetime
import requests
from io import StringIO
from github import Github
from getpass import getpass
from datetime import datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 29.4 MB/s eta 0:00:00


***TABELA REVIEWS***

In [2]:
def remove_acentos_e_especiais(texto):
    """Remove acentos e caracteres especiais do texto"""
    if pd.isna(texto):
        return texto
    texto = unicodedata.normalize('NFKD', str(texto)).encode('ASCII', 'ignore').decode('ASCII')
    return ''.join(c for c in texto if c.isalnum() or c.isspace())

def conver_data(data_str):
    """Converte data no formato AAAAMMDD para DD/MM/AAAA"""
    try:
        return datetime.strptime(data_str, '%Y%m%d').strftime('%d/%m/%Y')
    except:
        return data_str

def main():
    github_url = "https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/bronze/olist_order_reviews_dataset.csv"

    try:
        response = requests.get(github_url)
        response.raise_for_status()

        df = pd.read_csv(StringIO(response.text))

        print("\nColunas identificadas no arquivo:")
        print(df.columns.tolist())

        for coluna in df.columns:
            if df[coluna].dtype == 'object':
                df[coluna] = df[coluna].apply(remove_acentos_e_especiais)
                df[coluna] = df[coluna].str.upper().str.strip()
                df[coluna] = df[coluna].str.replace(r'\s+', ' ', regex=True)

        if 'review_creation_date' in df.columns:
            df['review_creation_date'] = df['review_creation_date'].astype(str).str.split().str[0]
            df['review_creation_date'] = df['review_creation_date'].apply(conver_data)
            print("\nData convertida (DD/MM/AAAA):")
            print(df['review_creation_date'].head())

        if 'review_answer_timestamp' in df.columns:
            df.drop('review_answer_timestamp', axis=1, inplace=True)
            print("\nColuna 'review_answer_timestamp' removida com sucesso!")

        df = df.dropna().drop_duplicates()

        output_filename = 'order_reviews_tratados.csv'
        df.to_csv(output_filename, index=False, encoding='utf-8')

        print("\nProcesso concluído com sucesso!")
        print("Dados finais disponíveis em:", output_filename)
        print("Colunas finais:", df.columns.tolist())

        try:
            GITHUB_TOKEN = getpass("🔑 Digite seu token do GitHub e pressione Enter: ")
            g = Github(GITHUB_TOKEN)

            repo_name = "Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251"
            repo = g.get_repo(repo_name)

            github_file_path = "projeto/development/silver/order_reviews_tratados.csv"

            with open(output_filename, "r") as file:
                content = file.read()

            try:
                contents = repo.get_contents(github_file_path)
                repo.update_file(
                    path=github_file_path,
                    message="Atualização automática via Colab",
                    content=content,
                    sha=contents.sha,
                    branch="main"
                )
                print("Arquivo atualizado no GitHub!")
            except Exception as e:
                if "404" in str(e):
                    repo.create_file(
                        path=github_file_path,
                        message="Upload via Colab",
                        content=content,
                        branch="main"
                    )
                    print("☁️ Arquivo criado no GitHub!")
                else:
                    raise e

        except Exception as e:
            print(f"\nErro ao enviar para o GitHub: {e}")

    except Exception as e:
        print(f"\nErro ao processar o arquivo: {e}")

if __name__ == "__main__":
    main()


Colunas identificadas no arquivo:
['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']

Data convertida (DD/MM/AAAA):
0    18/01/2018
1    10/03/2018
2    17/02/2018
3    21/04/2017
4    01/03/2018
Name: review_creation_date, dtype: object

Coluna 'review_answer_timestamp' removida com sucesso!

Processo concluído com sucesso!
Dados finais disponíveis em: order_reviews_tratados.csv
Colunas finais: ['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date']


KeyboardInterrupt: Interrupted by user

***TABELA ORDER_ITEMS_ID***

In [ ]:
def remove_acentos_e_especiais(texto):
    """Remove acentos e caracteres especiais do texto"""
    if pd.isna(texto):
        return texto
    texto = str(texto)
    texto = unicodedata.normalize('NFD', texto)
    texto = texto.encode('ascii', 'ignore').decode('utf-8')
    return texto

def main():

    github_url = "https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/bronze/olist_order_items_dataset.csv"

    try:
        response = requests.get(github_url)
        response.raise_for_status()

        df = pd.read_csv(StringIO(response.text))

        print("\nColunas identificadas no arquivo:")
        print(df.columns.tolist())

        for coluna in df.columns:
            if df[coluna].dtype == 'object':
                df[coluna] = df[coluna].apply(remove_acentos_e_especiais)
                df[coluna] = df[coluna].str.upper().str.strip()
                df[coluna] = df[coluna].str.replace(r'\s+', ' ', regex=True)

        if 'shipping_limit_date' in df.columns:
            df.drop('shipping_limit_date', axis=1, inplace=True)
            print("\nColuna 'shipping_limit_date' removida com sucesso!")

        df = df.dropna().drop_duplicates()

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_filename = f'order_items_tratados_{timestamp}.csv'
        df.to_csv(output_filename, index=False, encoding='utf-8')

        print("\nProcesso concluído com sucesso!")
        print("Dados finais disponíveis em:", output_filename)
        print("Colunas finais:", df.columns.tolist())

        try:
            GITHUB_TOKEN = getpass("🔑 Digite seu token do GitHub e pressione Enter: ")
            g = Github(GITHUB_TOKEN)

            repo_name = "Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251"
            repo = g.get_repo(repo_name)

            github_file_path = f"projeto/development/silver/order_items_tratados.csv"

            with open(output_filename, "r") as file:
                content = file.read()

            repo.create_file(
                path=github_file_path,
                message=f"Atualização automática via Colab",
                content=content,
                branch="main"
            )
            print("☁️ Arquivo criado no GitHub!")

        except Exception as e:
            print(f"\nErro ao enviar para o GitHub: {e}")

    except Exception as e:
        print(f"\nErro ao processar o arquivo: {e}")

if __name__ == "__main__":
    main()

***TABELA ORDER_PAYMENTS***

In [ ]:
def remove_acentos_e_especiais(texto):
    """Remove acentos e caracteres especiais do texto"""
    if pd.isna(texto):
        return texto
    texto = unicodedata.normalize('NFKD', str(texto)).encode('ASCII', 'ignore').decode('ASCII')
    return ''.join(c for c in texto if c.isalnum() or c.isspace())

def main():
    github_url = "https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/bronze/olist_order_payments_dataset.csv"
    response = requests.get(github_url)
    response.raise_for_status()
    df = pd.read_csv(StringIO(response.text))

    for coluna in df.columns:
        if df[coluna].dtype == 'object':
            df[coluna] = df[coluna].apply(remove_acentos_e_especiais)
            df[coluna] = df[coluna].str.upper().str.strip()
            df[coluna] = df[coluna].str.replace(r'\s+', ' ', regex=True)

    if 'payment_sequential' in df.columns:
        df.drop('payment_sequential', axis=1, inplace=True)

    df = df.dropna().drop_duplicates()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"order_payments_tratados_{timestamp}.csv"
    df.to_csv(output_filename, index=False)

    print("\nProcesso concluído com sucesso!")
    print("Dados finais disponíveis em:", output_filename)
    print("Colunas finais:", df.columns.tolist())

    try:
        GITHUB_TOKEN = getpass("🔑 Digite seu token do GitHub e pressione Enter: ")
        g = Github(GITHUB_TOKEN)

        repo_name = "Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251"
        repo = g.get_repo(repo_name)

        github_file_path = "projeto/development/silver/order_payments_tratados.csv"

        with open(output_filename, "r") as file:
            content = file.read()

        repo.create_file(
            path=github_file_path,
            message=f"Atualização automática via Colab",
            content=content,
            branch="main"
        )
        print("☁️ Arquivo criado no GitHub!")

    except Exception as e:
        print(f"\nErro ao enviar para o GitHub: {e}")

if __name__ == "__main__":
    main()

***TABELA ORDER_PRODUCTS***

In [ ]:
def remove_acentos_e_especiais(texto):
    """Remove acentos e caracteres especiais do texto"""
    if pd.isna(texto):
        return texto
    texto = unicodedata.normalize('NFKD', str(texto)).encode('ASCII', 'ignore').decode('ASCII')
    return ''.join(c for c in texto if c.isalnum() or c.isspace())

def main():
    github_url = "https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/bronze/olist_products_dataset.csv"
    response = requests.get(github_url)
    response.raise_for_status()
    df = pd.read_csv(StringIO(response.text))

    print("\nColunas identificadas no arquivo:")
    print(df.columns.tolist())

    colunas_para_remover = [
        "product_name_lenght",
        "product_description_lenght",
        "product_photos_qty",
        "product_weight_g",
        "product_length_cm",
        "product_height_cm",
        "product_width_cm"
    ]

    colunas_removidas = [col for col in colunas_para_remover if col in df.columns]
    df.drop(columns=colunas_removidas, errors='ignore', inplace=True)

    if colunas_removidas:
        print("\nColunas removidas com sucesso:", colunas_removidas)
    else:
        print("\nNenhuma das colunas especificadas foi encontrada no arquivo.")

    for coluna in df.columns:
        if df[coluna].dtype == 'object':
            df[coluna] = df[coluna].apply(remove_acentos_e_especiais)
            df[coluna] = df[coluna].str.upper().str.strip()
            df[coluna] = df[coluna].str.replace(r'\s+', ' ', regex=True)

    df = df.dropna().drop_duplicates()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f'order_products_tratados_{timestamp}.csv'
    df.to_csv(output_filename, index=False, encoding='utf-8')

    print("\nProcesso concluído com sucesso!")
    print("Dados finais disponíveis em:", output_filename)
    print("Colunas finais:", df.columns.tolist())

    try:
        GITHUB_TOKEN = getpass.getpass("🔑 Digite seu token do GitHub e pressione Enter: ")
        g = Github(GITHUB_TOKEN)

        repo_name = "Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251"
        repo = g.get_repo(repo_name)

        github_file_path = "projeto/development/silver/order_products_tratados.csv"

        try:
            contents = repo.get_contents(github_file_path, ref="main")
            repo.update_file(
                path=github_file_path,
                message=f"Atualização automática via Colab",
                content=df.to_csv(index=False, encoding='utf-8'),
                sha=contents.sha,
                branch="main"
            )
            print("☁️ Arquivo atualizado no GitHub com sucesso!")
        except:
            repo.create_file(
                path=github_file_path,
                message=f"Criação automática ({timestamp})",
                content=df.to_csv(index=False, encoding='utf-8'),
                branch="main"
            )
            print("☁️ Arquivo criado no GitHub!")

    except Exception as e:
        print(f"\nErro ao enviar para o GitHub: {str(e)}")

if __name__ == "__main__":
    main()

NameError: name 'requests' is not defined

***TABELA ORDER_SELLERS***

In [ ]:
def remove_acentos_e_especiais(texto):
    """Remove acentos e caracteres especiais do texto"""
    if pd.isna(texto):
        return texto
    texto = unicodedata.normalize('NFKD', str(texto)).encode('ASCII', 'ignore').decode('ASCII')
    return ''.join(c for c in texto if c.isalnum() or c.isspace())

def main():
    github_url = "https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/bronze/olist_sellers_dataset.csv"
    response = requests.get(github_url)
    response.raise_for_status()
    df = pd.read_csv(StringIO(response.text))

    print("\nColunas identificadas no arquivo:")
    print(df.columns.tolist())

    if 'seller_zip_code_prefix' in df.columns:
        df.drop('seller_zip_code_prefix', axis=1, inplace=True)
        print("\nColuna 'seller_zip_code_prefix' removida com sucesso!")

    colunas_para_remover = [
    ]

    colunas_removidas = [col for col in colunas_para_remover if col in df.columns]
    df.drop(columns=colunas_removidas, errors='ignore', inplace=True)

    if colunas_removidas:
        print("\nColunas removidas com sucesso:", colunas_removidas)
    else:
        print("\nNenhuma das colunas especificadas foi encontrada no arquivo.")

    for coluna in df.columns:
        if df[coluna].dtype == 'object':
            df[coluna] = df[coluna].apply(remove_acentos_e_especiais)
            df[coluna] = df[coluna].str.upper().str.strip()
            df[coluna] = df[coluna].str.replace(r'\s+', ' ', regex=True)

    df = df.dropna().drop_duplicates()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f'order_sellers_tratados_{timestamp}.csv'
    df.to_csv(output_filename, index=False, encoding='utf-8')

    print("\nProcesso concluído com sucesso!")
    print("Dados finais disponíveis em:", output_filename)
    print("Colunas finais:", df.columns.tolist())

    try:
        GITHUB_TOKEN = getpass.getpass("🔑 Digite seu token do GitHub e pressione Enter: ")
        g = Github(GITHUB_TOKEN)

        repo_name = "Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251"
        repo = g.get_repo(repo_name)

        github_file_path = "projeto/development/silver/order_sellers_tratados.csv"

        try:
            contents = repo.get_contents(github_file_path, ref="main")
            repo.update_file(
                path=github_file_path,
                message=f"Atualização automática via Colab",
                content=df.to_csv(index=False, encoding='utf-8'),
                sha=contents.sha,
                branch="main"
            )
            print("☁️ Arquivo atualizado no GitHub com sucesso!")
        except:
            repo.create_file(
                path=github_file_path,
                message=f"Criação automática ({timestamp})",
                content=df.to_csv(index=False, encoding='utf-8'),
                branch="main"
            )
            print("☁️ Arquivo criado no GitHub!")

    except Exception as e:
        print(f"\nErro ao enviar para o GitHub: {str(e)}")

if __name__ == "__main__":
    main()

***TABELA ORDERS***

In [ ]:
def remove_acentos_e_especiais(texto):
    """Remove acentos e caracteres especiais do texto"""
    if pd.isna(texto):
        return texto
    texto = str(texto)
    texto = unicodedata.normalize('NFD', texto)
    texto = texto.encode('ascii', 'ignore').decode('utf-8')
    return texto

def main():
    github_url = "https://raw.githubusercontent.com/Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251/main/projeto/development/bronze/olist_orders_dataset.csv"
    response = requests.get(github_url)
    response.raise_for_status()
    df = pd.read_csv(StringIO(response.text))

    print("\nColunas identificadas no arquivo:")
    print(df.columns.tolist())

    for coluna in df.columns:
        if df[coluna].dtype == 'object':
            df[coluna] = df[coluna].apply(remove_acentos_e_especiais)
            df[coluna] = df[coluna].str.upper().str.strip()
            df[coluna] = df[coluna].str.replace(r'\s+', ' ', regex=True)

    colunas_data = [
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date",
    ]

    for coluna in colunas_data:
        if coluna in df.columns:
            df[coluna] = df[coluna].str.split().str[0]  # Pega apenas a parte da data
            df[coluna] = pd.to_datetime(df[coluna], errors='coerce').dt.strftime('%d/%m/%Y')
            print(f"✔ Coluna {coluna} formatada para DD/MM/AAAA")

    colunas_para_remover = [
        "product_name_lenght",
        "product_description_lenght",
        "product_photos_qty",
        "product_weight_g",
        "product_length_cm",
        "product_height_cm",
        "product_width_cm"
    ]

    colunas_removidas = [col for col in colunas_para_remover if col in df.columns]
    df.drop(columns=colunas_removidas, inplace=True, errors='ignore')

    if colunas_removidas:
        print("\nColunas removidas:", ", ".join(colunas_removidas))
    else:
        print("\nNenhuma das colunas para remoção foi encontrada")

    df = df.dropna().drop_duplicates()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f'orders_tratados_{timestamp}.csv'
    df.to_csv(output_filename, index=False, encoding='utf-8')

    print("\nProcesso concluído com sucesso!")
    print("Dados finais disponíveis em:", output_filename)
    print("Colunas finais:", df.columns.tolist())

    try:
        GITHUB_TOKEN = getpass.getpass("🔑 Digite seu token do GitHub e pressione Enter: ")
        g = Github(GITHUB_TOKEN)

        repo_name = "Tecnologia-em-Banco-de-Dados-PUC-Minas/eixo5_grupo2_20251"
        repo = g.get_repo(repo_name)

        github_file_path = "projeto/development/silver/orders_tratados.csv"

        try:
            contents = repo.get_contents(github_file_path, ref="main")
            repo.update_file(
                path=github_file_path,
                message=f"Atualização automática via Colab",
                content=df.to_csv(index=False, encoding='utf-8'),
                sha=contents.sha,
                branch="main"
            )
            print("☁️ Arquivo atualizado no GitHub com sucesso!")
        except:
            repo.create_file(
                path=github_file_path,
                message=f"Criação automática ({timestamp})",
                content=df.to_csv(index=False, encoding='utf-8'),
                branch="main"
            )
            print("☁️ Arquivo criado no GitHub!")

    except Exception as e:
        print(f"\nErro ao enviar para o GitHub: {str(e)}")

if __name__ == "__main__":
    main()